In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from scipy.stats import linregress
from scipy.stats import norm
from operator import itemgetter
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Position,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,L. Messi,31,94,94,€110.5M,€565K,5.0,4.0,4.0,RF,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,Cristiano Ronaldo,33,94,94,€77M,€405K,5.0,4.0,5.0,ST,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,Neymar Jr,26,92,93,€118.5M,€290K,5.0,5.0,5.0,LW,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,De Gea,27,91,93,€72M,€260K,4.0,3.0,1.0,GK,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,K. De Bruyne,27,91,92,€102M,€355K,4.0,5.0,4.0,RCM,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


# Data Sanitization

Some columns were removed in Microsoft Excel that were trivially unrelated (e.g. photos, country of origin, preferred foot). 

Before going any further, we must:
1. Turn values/wages to floats
2. Divide goalkeepers and other players (They have different related scores)
3. Calculate position scores
4. Relate/Combine value and wage. The response variable for this study is some mix between the value and the wage, the formula for which we need to figure out.

In [3]:
# 1
def priceToFloat(price):
    price = price[1:]
    try:
        price = float(price)
    except ValueError:
        price = float(price[:-1]) * (1000000 if price[-1] == 'M' else 1000 if price[-1] == 'K' else 1)
    return price
df['Value'] = df['Value'].apply(lambda x: priceToFloat(x))
df['Wage'] = df['Wage'].apply(lambda x: priceToFloat(x))

In [4]:
# 2
goalies_df = df[df['Position'] == 'GK'].dropna(axis=1)
players_df = df[df['Position'] != 'GK'].dropna()

In [5]:
# 3
def calculateScore(equation):
    return sum(map(int,(equation).split('+')))
for column in range(10,36):
    players_df[players_df.columns[column]] = players_df[players_df.columns[column]].apply(lambda x: calculateScore(x))